In [ ]:
import sys
import math
import random
import copy

import pandas as pd
import numpy as np

from tqdm import tqdm
import time

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import animation as animation

import matplotlib as mpl

import os

from PIL import Image, ImageSequence
from skimage.transform import resize

import json

directory = '' # your directory

# from google.colab import drive
# drive.mount('/content/drive')

#### data

In [ ]:
###################################################################################
# training data Ferguson_2018

forest = Image.open(directory + 'Data/land_data_CA/Ferguson/canopy_Ferguson_2018.tif')

density = Image.open(directory + 'Data/land_data_CA/Ferguson/density_Ferguson_2018.tif')

altitude = Image.open(directory + 'Data/land_data_CA/Ferguson/slope_Ferguson_2018.tif')

###################################################################################
# train data Bear_2020

# forest = Image.open(directory + 'Data/land_data_CA/Bear/canopy_Bear_2020.tif')

# altitude = Image.open(directory + 'Data/land_data_CA/Bear/slope_Bear_2020.tif')

# density = Image.open(directory + 'Data/land_data_CA/Bear/density_Bear_2020.tif')

###################################################################################
#train data Chimney_2016

# forest = Image.open(directory + 'Data/land_data_CA/Chimney/canopy_Chimney_2016.tif')

# altitude = Image.open(directory + 'Data/land_data_CA/Chimney/slope_Chimney_2016.tif')

# density = Image.open(directory + 'Data/land_data_CA/Chimney/density_Chimney_2016.tif')


In [ ]:
forest = Image.open(directory + 'Data/land_data_CA/Ferguson/canopy_Ferguson_2018.tif')
forest = np.array(forest)
forest[forest<-999.] = 0.
forest = forest/np.max(forest)

forest = resize(forest, (128, 128))

#### Simulation

In [ ]:
def resize_func(ori_forest, ori_altitude, ori_density, size):
    resized_forest = resize(ori_forest, (size, size))
    resized_altitude = resize(ori_altitude, (size, size))
    resized_density = resize(ori_density, (size, size))
    resized_density = np.round(resized_density/np.max(resized_density))
    return  resized_forest, resized_altitude, resized_density


In [ ]:
def random_ignition(dim_x, dim_y):
    """set up different ignition on map, here is random in the field around the center area"""
    field = np.ones((dim_x, dim_y))*2
    x = random.randint(30, dim_x-30)
    y = random.randint(30, dim_y-30)

    for i in range(x, x+4):
        for j in range(y, y+4):
            field[i, j] = 3
    return x, y, field

def centre_ignition(dim_x, dim_y):
    """set up different ignition on map, here is the center area"""
    field = np.ones((dim_x, dim_y)) * 2
    x = round(dim_x/2)
    y = round(dim_y/2)
    for i in range(x, x+3):
        for j in range(y, y+3):
            field[i, j] = 3
    return field

In [ ]:
def colormap(i, array):
    np_array = np.array(array)
    plt.imshow(np_array)#, interpolation="none")#, cmap=cm.plasma)
    plt.title(i)
    plt.colorbar()
    plt.axis('off')
    plt.show()

def print_forest(forest):
    for i in range(n_row):
        for j in range(n_col):
            sys.stdout.write(str(forest[i][j]))
        sys.stdout.write("\n")

In [ ]:
def init_vegetation():
    veg_matrix = [[0 for col in range(n_col)] for row in range(n_row)]
    for i in range(n_row):
        for j in range(n_col):
            veg_matrix[i][j] = 1
    return veg_matrix

def init_density():
    den_matrix = [[0 for col in range(n_col)] for row in range(n_row)]
    for i in range(n_row):
        for j in range(n_col):
            den_matrix[i][j] = 1.0
    return den_matrix

def init_altitude():
    alt_matrix = [[0 for col in range(n_col)] for row in range(n_row)]
    for i in range(n_row):
        for j in range(n_col):
            alt_matrix[i][j] = 1
    return alt_matrix

def init_forest():
    forest = [[0 for col in range(n_col)] for row in range(n_row)]
    for i in range(n_row):
        for j in range(n_col):
            forest[i][j] = 2
    ignite_col = int(n_col//2)
    ignite_row = int(100)
    for row in range(ignite_row-1, ignite_row+1):
        for col in range(ignite_col-1,ignite_col+1):
            forest[row][col] = 3
    return forest

In [ ]:
def tg(x):
    return math.degrees(math.atan(x))

In [ ]:
def get_slope(altitude_matrix):
    slope_matrix = [[0 for col in range(n_col)] for row in range(n_row)]
    for row in range(n_row):
        for col in range(n_col):
            sub_slope_matrix = [[0,0,0],[0,0,0],[0,0,0]]
            if row == 0 or row == n_row-1 or col == 0 or col == n_col-1:  # margin is flat
                slope_matrix[row][col] = sub_slope_matrix
                continue
            current_altitude = altitude_matrix[row][col]
            sub_slope_matrix[0][0] = tg((current_altitude - altitude_matrix[row-1][col-1])/1.414)
            sub_slope_matrix[0][1] = tg(current_altitude - altitude_matrix[row-1][col])
            sub_slope_matrix[0][2] = tg((current_altitude - altitude_matrix[row-1][col+1])/1.414)
            sub_slope_matrix[1][0] = tg(current_altitude - altitude_matrix[row][col-1])
            sub_slope_matrix[1][1] = 0
            sub_slope_matrix[1][2] = tg(current_altitude - altitude_matrix[row][col+1])
            sub_slope_matrix[2][0] = tg((current_altitude - altitude_matrix[row+1][col-1])/1.414)
            sub_slope_matrix[2][1] = tg(current_altitude - altitude_matrix[row+1][col])
            sub_slope_matrix[2][2] = tg((current_altitude - altitude_matrix[row+1][col+1])/1.414)
            slope_matrix[row][col] = sub_slope_matrix
    return slope_matrix

In [ ]:
def simulation(stage, per_bar_time_1, temp_bar_time_1, per_bar_time_2=None, per_bar_time_3=None, temp_bar_time_2=None, temp_bar_time_3=None, ignition='center',
               ig_field=None, rand_wind=False, make_fig=False, forest_before_bar=None, whole_time=None, per_bar_input_1=None, per_bar_input_2=None,
               per_bar_input_3=None, temp_bar_input_1=None, temp_bar_input_2=None, temp_bar_input_3=None, max_per_len=None, max_temp_len=None, both_pt=False):
    """
    input:
    stage = before bar:1, after bar:2
    bar_time = when did the bar was set
    investigate_time = how long the investigate time be (after set the bar)
    bar_input = matrix of bar
    max_per_len = the maxmimum length of the permanent bar
    max_temp_len = the maxmimum length of the temporary bar

    output:
        stage1: the list of forest matrix before bar in time sequence
        stage2: the list of forest matrix after bar in time sequence
    """
    ###########################################################################
    #init

    V = 5. # need to find the true wind data
    p_h = 0.58
    a = 0.078
    c_1 = 0.045
    c_2 = 0.131

    if ignition == 'center':
        ignition = centre_ignition(np.array(forest).shape[0], np.array(forest).shape[1])
    elif ignition == 'random':
        ignition = ig_field

    ##############################################################################
    def calc_pw(theta,c_1,c_2,V):
        t = math.radians(theta)
        ft = math.exp(V*c_2*(math.cos(t)-1))
        return math.exp(c_1*V)*ft

    def count_3_and_4(matrix):
        """count the ignited fire points"""
        count = 0
        for row in matrix:
            for element in row:
                if element == 3 or element == 4:
                    count += 1
        return count

    def get_wind():
        wind_matrix = [[1,0.5,0],
                  [0.5,0,0],
                  [0,0,0]]
        return wind_matrix

    def rand_wind():

        wind_list = [[[1,0.5,0],
                  [0.5,0,0],
                  [0,0,0]],
                  [[0.5,1,0.5],
                  [0,0,0],
                  [0,0,0]],
                  [[0,0.5,1],
                  [0,0,0.5],
                  [0,0,0]],
                  [[0,0,0.5],
                  [0,0,1],
                  [0,0,0.5]],
                  [[0,0,0],
                  [0,0,0.5],
                  [0,0.5,1]],
                  [[0,0,0],
                  [0.5,1,0.5],
                  [0,0,0]],
                  [[0,0,0],
                  [0.5,0,0],
                  [1,0.5,0]],
                  [[0.5,0,0],
                  [1,0,0],
                  [0.5,0,0]]]

        return random.choice(wind_list)

    def burn_or_not_burn(abs_row, abs_col, neighbour_matrix, p_h, a):
        p_veg = vegetation_matrix[abs_row][abs_col]
        p_den = {0:-0.4,1:0,2:0.3}[density_matrix[abs_row][abs_col]]
        for row in [0,1,2]:
            for col in [0,1,2]:
                if neighbour_matrix[row][col] == 3: # we only care if there is a neighbour that is burning
                    slope = slope_matrix[abs_row][abs_col][row][col]
                    p_slope = math.exp(a * slope)
                    p_wind = wind_matrix[row][col]
                    p_burn = p_h * (0.5 + p_veg*10.) * (1 + p_den) * p_wind * p_slope
                    if p_burn > random.random():
                        return 3  #start burning
        return 2 # not burn

    def permanant_burn_or_not_burn(abs_row, abs_col, neighbour_matrix, p_h, a, p_bar=0.05):
        # calculate whether the fire will pass if the neighbour is a permanent bar
        p_veg = vegetation_matrix[abs_row][abs_col]
        p_den = {0:-0.4,1:0,2:0.3}[density_matrix[abs_row][abs_col]]
        for row in [0,1,2]:
            for col in [0,1,2]:
                if neighbour_matrix[row][col] == 3:
                    slope = slope_matrix[abs_row][abs_col][row][col]
                    p_slope = math.exp(a * slope)
                    p_wind = wind_matrix[row][col]
                    p_burn = p_h * (0.5 + p_veg*10.) * (1 + p_den) * p_wind * p_slope * p_bar # multiply one more prob p_bar
                    if p_burn > random.random():
                        return 3  #start burning
        return 5 # not burning

    def update_forest(old_forest, before_bar_forest):
        result_forest = [[1 for i in range(n_col)] for j in range(n_row)]
        for row in range(1, n_row-1):
            for col in range(1, n_col-1):

                if old_forest[row][col] == 1 or old_forest[row][col] == 4:
                    result_forest[row][col] = old_forest[row][col]  # no fuel or burnt down

                if old_forest[row][col] == 3: # burning
                    if random.random() < 0.4:
                        result_forest[row][col] = 3  # TODO need to change back here
                    else:
                        result_forest[row][col] = 4

                if old_forest[row][col] == 2: # can burn
                    neighbours = [[row_vec[col_vec] for col_vec in range(col-1, col+2)]
                                    for row_vec in old_forest[row-1:row+2]]
                    result_forest[row][col] = burn_or_not_burn(row, col, neighbours,p_h,a)

                if old_forest[row][col] == 5: # permanent bar
                    neighbours = [[row_vec[col_vec] for col_vec in range(col-1, col+2)]
                                    for row_vec in old_forest[row-1:row+2]]
                    result_forest[row][col] = permanant_burn_or_not_burn(row, col, neighbours, p_h,a)

                if old_forest[row][col] == 6: #t bar last day
                    if both_pt:
                        result_forest[row][col] = forest_after_pbar1[row][col]
                    else:
                        result_forest[row][col] = before_bar_forest[row][col]

                if old_forest[row][col] > 6: #t bar down grade
                    result_forest[row][col] = old_forest[row][col] - 1

        return result_forest

    #check_bar_type
    def check_temp_bar_input(forest, temp_bar_input, max_temp_len):
        temp_count = 0
        new_forest = forest

        for row in temp_bar_input:
            for element in row:
                if element != 0 and element < 6 and element > 15:
                    raise TypeError("The temp bar input is wrong, should be either 0 or 6.")

                #check_bar_length
                if element >= 6:
                    temp_count += 1

        if  temp_count > max_temp_len:
            raise TypeError("The max length of temp bar is exceeded.")


    def check_per_bar_input(forest, per_bar_input, max_per_len):
        per_count = 0
        new_forest = forest

        for row in per_bar_input:
            for element in row:
                if element != 0 and element != 5 :
                    raise TypeError("The per bar input is wrong, should be either 0 or 5.")

                #check_bar_length
                if element == 5:
                    per_count += 1

        if per_count > max_per_len :
            raise TypeError("The max length of per bar is exceed.")

        #check permanent bar distance from the fire front
        for row in range(5, n_row-5):
            for col in range(5, n_col-5):
                neighbours = [[row_vec[col_vec] for col_vec in range(col-5, col+6)]
                            for row_vec in new_forest[row-5:row+6]]
                if per_bar_input[row][col] == 5 and any(3 in row for row in neighbours):
                    raise TypeError("The permenant bar is too close to the fire front.")

    ###########################################################
    # custormize colorbar

    cmap = mpl.colors.ListedColormap(['orange', 'yellow', 'green', 'red', 'black', 'white', 'blue'])
    cmap.set_over('0.25')
    cmap.set_under('0.75')
    bounds = [1.0, 2.02, 2.27, 3.01, 3.5, 5.1, 5.9, 6.9]
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

    ############################################################
    #init
    fields_1_sim = np.zeros((1,100))

    vegetation_matrix = forest

    density_matrix = density.tolist()

    altitude_matrix = altitude.tolist()

    if rand_wind:
        wind_matrix = rand_wind()
    else:
        wind_matrix = get_wind()

    new_forest = ignition.tolist()

    slope_matrix = get_slope(altitude_matrix)

    ############################################################
    #the fire progress before set the bar
    if stage == 1:
        #############################################################################
        #init
        stage_1_forest_list = [np.array(new_forest)]

        #############################################################################
        #begin to loop
        bar_time = min(temp_bar_time_1, per_bar_time_1)

        for day in range(bar_time):
            new_forest = copy.deepcopy(update_forest(new_forest, None))
            forest_array = np.array(new_forest)
            stage_1_forest_list.append(forest_array)

            if day>0 and day%1 == 0 and day < bar_time and make_fig:

                plt.imshow(forest + forest_array,  cmap = cmap, norm = norm, interpolation="none" )
                plt.axis('off')
                plt.title(f'temp_1:{temp_bar_time_1}, temp_2:{temp_bar_time_2}, temp_3:{temp_bar_time_3}, per_1:{per_bar_time_1}, per_2:{per_bar_time_2}, per_3:{per_bar_time_3}, time step: {day}')

                # Create the directory if it doesn't exist
                if not os.path.exists(directory):
                    os.makedirs(directory)
                # Save the image with the desired file path
                plt.savefig(directory + 'fire_' + str(day).zfill(3) + '.png', format='png', bbox_inches='tight')

                plt.show()
                plt.close()
                print('burning', np.sum(forest_array==4))

            plt.show()
            plt.close()

        return stage_1_forest_list

    ############################################################
    #the fire progress after set the bar
    elif stage == 2:
        ############################################################################
        #init
        new_forest = copy.deepcopy(forest_before_bar)
        loss = count_3_and_4(new_forest)
        stage_2_forest_list = []

        ############################################################################
        #update forest
        for day in range(min(temp_bar_time_1, per_bar_time_1), whole_time):

            if day == temp_bar_time_1: #store forest before temp bar
                forest_before_temp_bar = copy.deepcopy(new_forest)

                check_temp_bar_input(new_forest, temp_bar_input_1, max_temp_len) #check input
                #insert bar
                for i in range(n_row):
                    for j in range(n_col):
                        if temp_bar_input_1[i][j] != 0:
                            new_forest[i][j] = temp_bar_input_1[i][j]

            if day == temp_bar_time_2:
                check_temp_bar_input(new_forest, temp_bar_input_2, max_temp_len) #check input
                #insert bar
                for i in range(n_row):
                    for j in range(n_col):
                        if temp_bar_input_2[i][j] != 0:
                            new_forest[i][j] = temp_bar_input_2[i][j]

            if day == temp_bar_time_3:
                check_temp_bar_input(new_forest, temp_bar_input_3, max_temp_len) #check input
                #insert bar
                for i in range(n_row):
                    for j in range(n_col):
                        if temp_bar_input_3[i][j] != 0:
                            new_forest[i][j] = temp_bar_input_3[i][j]

            if day == per_bar_time_1:
                check_per_bar_input(new_forest, per_bar_input_1, max_per_len) #check input

                #insert bar
                for i in range(n_row):
                    for j in range(n_col):
                        if per_bar_input_1[i][j] != 0:
                            new_forest[i][j] = per_bar_input_1[i][j] #save the forest after pbar1
                if both_pt:
                    forest_after_pbar1=copy.deepcopy(new_forest)

            if day == per_bar_time_2:
                check_per_bar_input(new_forest, per_bar_input_2, max_per_len) #check input
                #insert bar
                for i in range(n_row):
                    for j in range(n_col):
                        if per_bar_input_2[i][j] != 0:
                            new_forest[i][j] = per_bar_input_2[i][j]

            if day == per_bar_time_3:
                check_per_bar_input(new_forest, per_bar_input_3, max_per_len) #check input
                #insert bar
                for i in range(n_row):
                    for j in range(n_col):
                        if per_bar_input_3[i][j] != 0:
                            new_forest[i][j] = per_bar_input_3[i][j]

            forest_array = np.array(new_forest)
            if day>0 and day%1 == 0 and make_fig:

                plt.imshow(forest + forest_array,  cmap = cmap, norm = norm, interpolation="none" )
                plt.axis('off')
                plt.title(f'temp_1:{temp_bar_time_1}, temp_2:{temp_bar_time_2}, temp_3:{temp_bar_time_3}, per_1:{per_bar_time_1}, per_2:{per_bar_time_2}, per_3:{per_bar_time_3}, time step: {day}')

                plt.show()
                plt.close()
                print('burning',np.sum(forest_array==4))

            plt.show()
            plt.close()

            new_forest = copy.deepcopy(update_forest(new_forest, forest_before_bar))
            forest_array = np.array(new_forest)
            stage_2_forest_list.append(forest_array)

        loss = count_3_and_4(new_forest) - loss
        return loss, stage_2_forest_list

    else:
        raise TypeError('The stage should be either 1 or 2')

In [ ]:
def create_tilt_bar_matrix2(input_x, input_y, angle, t_or_p):
    matrix = np.zeros((128, 128))

    m = np.tan(angle * np.pi /180)

    try:

        for y in range(input_y-8, input_y+8):

            # Calculate the x using y
            x = (y - input_y)/m + input_x

            # Draw the tilt bar in the matrix
            if t_or_p =='p':
                matrix[int(x), y] = 5
                matrix[int(x)+1, y] = 5
                matrix[int(x)-1, y] = 5

            elif t_or_p =='t':
                # we put in 16 since the second we input t bar, it will be down grade to 15.
                # To make t bar last for 10 steps, we need to input t bar's first set up be 16
                matrix[int(x), y] = 16
                matrix[int(x)+1, y] = 16
                matrix[int(x)-1, y] = 16


    except Exception:
        #print('Error m: ', m, '\tError y_input: ', input_y, '\tError angle: ', angle)
        pass

    return matrix

def make_rand_tilt_n_tbar_m_pbar(n,m,ig_x, ig_y, big_ig = False):

    while True:
        bars_list = []

        #make n tbar
        for i in range(n):
            tilt_angle = random.randint(30, 150)
            tilt_x = random.randint(ig_x-5, ig_x+5)
            tilt_y = random.randint(ig_y-5, ig_y+5)
            bars_list.append(create_tilt_bar_matrix2(tilt_x, tilt_y, tilt_angle, t_or_p='t'))

        #make m pbar
        if big_ig:
            for j in range(m):
                tilt_angle = random.randint(30,150)
                tilt_x = random.randint(5,123)
                tilt_y = random.randint(5,123)
                bars_list.append(create_tilt_bar_matrix2(tilt_x, tilt_y, tilt_angle, t_or_p='p'))

        else:
            for j in range(m):
                tilt_angle = random.randint(30,150)
                tilt_x = random.randint(ig_x-15,ig_x+20)
                tilt_y = random.randint(ig_y-15,ig_y+20)
                bars_list.append(create_tilt_bar_matrix2(tilt_x, tilt_y, tilt_angle, t_or_p='p'))

        if check_pt_bar(n,m,bars_list): #check validility
            return bars_list
            break


# def make_rand_tilt_n_tbar_m_pbar(n, m, ig_x, ig_y, ig=None):
#     def is_valid_distance(x, y, valid_positions, min_dist=5, max_dist=20):
#         # Check if (x, y) is within min_dist and max_dist from any position in valid_positions
#         for pos in valid_positions:
#             dist = np.sqrt((x - pos[0])**2 + (y - pos[1])**2)
#             if min_dist <= dist <= max_dist:
#                 return True
#         return False

#     while True:
#         bars_list = []

#         # Find positions of 3s in ig matrix
#         if ig is not None:
#             pos_3s = np.where(ig == 3)
#             positions_3s = list(zip(pos_3s[0], pos_3s[1]))

#         # Make n t-bars
#         for i in range(n):
#             tilt_angle = random.randint(30, 150)
#             tilt_x = random.randint(ig_x-5, ig_x+5)
#             tilt_y = random.randint(ig_y-5, ig_y+5)
#             bars_list.append(create_tilt_bar_matrix2(tilt_x, tilt_y, tilt_angle, 't'))

#         # Make m p-bars
#         for j in range(m):
#             while True:
#                 tilt_angle = random.randint(30, 150)
#                 tilt_x = random.randint(ig_x-15, ig_x+20)
#                 tilt_y = random.randint(ig_y-15, ig_y+20)

#                 # If ig is provided, ensure p-bar is set within the specified distance from any 3-valued cell
#                 if ig is not None and not is_valid_distance(tilt_x, tilt_y, positions_3s):
#                     continue
#                 break

#             bars_list.append(create_tilt_bar_matrix2(tilt_x, tilt_y, tilt_angle, 'p'))

#         if check_pt_bar(n, m, bars_list):  # Check validity
#             return bars_list


def check_pt_bar(N, M, bar_list):
    #check if bars are overlapping
    add_bar = np.zeros((n_col, n_row))
    overlap_list = []

    for bar_input in bar_list:
        add_bar += bar_input

    for n in range(N):
        for m in range(M):
            x = 16*(n+1) + 5*(m+1)
            overlap_list.append(x)

    if N >= 1:
        for n in range(1,N):
            x = 16*(n+1)
            overlap_list.append(x)

    if M >= 1:
        for m in range(1,M):
            x = 5*(m+1)
            overlap_list.append(x)

    for y in overlap_list:
        if y in add_bar: #check the bar is not overlap
            return False

    return True

In [ ]:
def change_state(forest_arr):
    """
    Change from CA state to ConvLSTM state.
    """
    # Define the conditions for each original state
    conditions = [
        forest_arr == 1, forest_arr == 2,
        forest_arr == 3, forest_arr == 4,
        forest_arr == 5, forest_arr == 6,
        forest_arr == 7, forest_arr == 8,
        forest_arr == 9, forest_arr == 10,
        forest_arr == 11, forest_arr == 12,
        forest_arr == 13, forest_arr == 14,
        forest_arr == 15
    ]

    # Define the new state corresponding to each condition, 0 for not burn (yet), 1 for burning or burnt, 2 for p bar, 3-12 for t bar
    choices = [0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

    # Apply the conditions and choices to the array
    forest_arr = np.select(conditions, choices, default=np.nan)

    # Check for any elements that didn't match any condition
    if np.isnan(forest_arr).any():
        raise TypeError("Unknown CA State detected in the array.")

    return forest_arr


In [ ]:
def setup_buring(forest):
    old_forest = copy.deepcopy(forest)
    rows, cols = np.shape(old_forest)

    for row in range(rows):
        for col in range(cols):

            if old_forest[row][col] == 1:
                old_forest[row][col] = 4

            if old_forest[row][col] == 0:
                old_forest[row][col] = 2

    for row in range(rows):
        for col in range(cols):

            if old_forest[row][col] == 4:

                neighbours = [[row_vec[col_vec] for col_vec in range(col-1, col+2)]
                                    for row_vec in old_forest[row-1:row+2]]

                if 2 in [int(x) for n_3 in neighbours for x in n_3]:
                    old_forest[row][col] = 3

    return old_forest

#### speed test

In [ ]:
import json

sizes = [128, 256, 384, 512, 640, 768]
fer_ca_time_dic = {}

for size in sizes:
    print(f'Now we are at size {size}')
    fer_ca_time_dic[f'{size}'] = []
    time_list = []

    # initialize
    forest = np.array(forest)
    forest[forest<-999.] = 0.
    forest = forest/np.max(forest)

    altitude = np.array(altitude)/np.max(altitude)

    density = np.array(density)
    density = np.round(density/np.max(density))

    #resize forest, altitude, density
    forest, altitude, density = resize_func(forest, altitude, density, size)

    density = np.round(density/np.max(density)) #double check: is this line neccesary

    n_row = forest.shape[0]
    n_col = forest.shape[1]

    no_bar = np.zeros((size, size))

    for n in tqdm(range(10)):
        forest_list = []

        while len(forest_list) < 2:

            start = time.time()

            num = len(forest_list)

            igx, igy, ig = random_ignition(np.array(forest).shape[0],np.array(forest).shape[1])

            bar_list = [no_bar] * 4

            stage1=simulation(stage=1, per_bar_time_1= 3, temp_bar_time_1= 2, ignition='random', ig_field=ig)

            try:
                stage2=simulation(stage=2, per_bar_time_1=3, temp_bar_time_1=2, per_bar_time_2=None, per_bar_time_3=None, temp_bar_time_2= 4, temp_bar_time_3= 6, ignition='random',
                        ig_field=ig, rand_wind=True, make_fig=False, forest_before_bar=stage1[-1], whole_time=25, per_bar_input_1=bar_list[3], per_bar_input_2=None,
                        per_bar_input_3=None, temp_bar_input_1=bar_list[0], temp_bar_input_2=bar_list[1], temp_bar_input_3=bar_list[2], max_per_len=50, max_temp_len=50)[1]
                forest_list.append(stage1+stage2)
                time_list.append(time.time()-start)

            except Exception as e:
                pass

    fer_ca_time_dic[f'{size}'] = time_list

    print(f'For {size}: {time_list}')
    # print('\n','size =',size, 'Inference time=', np.mean(mean_time_list))

print(fer_ca_time_dic)

In [ ]:
# Saving the dictionary to a file
with open(directory + 'Data/inference_speed_evaluation_data/fer_ca_time_dic.json', 'w') as json_file:
    json.dump(fer_ca_time_dic, json_file)
